# Zonal transport in the depth coordinate

This notebook is converted from diag_transport.py, which calculates the barotropic and baroclinic components of zonal transport.

## Last updated 9 October 2017.

In [1]:
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt

v45 - wdx1: output017 (y50)  wdx2: output024 (y65) wdx3: output031 (y80)

v57 - wdx1: output013 (y50)  wdx2: output020 (y65) wdx3: output027 (y80) 

v59 - wdx1: output013 (y50)  wdx2: output020 (y65) wdx3: output027 (y80) 

In [2]:
data  = nc.Dataset('/short/v45/lxy581/mom6/archive/so_mom6_v45/output031/prog.nc','r')
coord = nc.Dataset('/short/v45/lxy581/mom6/archive/so_mom6_v45/output031/Vertical_coordinate.nc','r')

In [3]:
yh = data.variables['yh'][:]
xq = data.variables['xq'][:]
zl = data.variables['zl'][:]
t  = data.variables['Time'][:]

Read layer thicknesses in the depth space in meters:

In [4]:
ds = coord.variables['ds'][:]

Horizontal resolution (km) in the Cartesian coordinate:

In [5]:
dx = 10.

In [6]:
nz = np.size(zl)
nx = np.size(xq)
ny = np.size(yh)
nt = np.size(t)

In [7]:
acc  = np.zeros((1,nz))
acct = np.full((1,nz),np.nan)
vel  = np.zeros((1,nz))
velt = np.full((1,nz),np.nan)

In [8]:
for k in xrange(0,nt):
    u  = data.variables['u'][k,:,:,:]  # 3D zonal velocity
    uz = np.nanmean(u,axis=2)          # zonal mean uz(z,y)
    uy = np.nansum(uz,axis=1)          # meridional sum uy(z)
    uh = uy*ds*dx*1.e+3                # zonal transport uh(z) in m3/s

    velt[0,:] = uy
    acct[0,:] = uh
    
    vel = np.append(vel,velt,0)
    acc = np.append(acc,acct,0)

The first dimension of variable vel (and acc) is time.
Variable velm (accm) is the time-and-zonal-mean zonal velocity (transport) as a function of depth. 

In [9]:
vel  = vel[1:,:]
acc  = acc[1:,:]
velm = np.nanmean(vel,axis=0) # velm(z)
accm = np.nanmean(acc,axis=0) # accm(z)

##Total transport (Sv)

In [10]:
acc_tt = np.nansum(accm)
print acc_tt/1.e+6

105.7777187


## Barotropic transport (Sv)

In [11]:
acc_bt = np.nansum(np.nanmin(velm)*ds*dx*1.e+3)
print acc_bt/1.e+6

4.68612556384


##Baroclinic transport (Sv)

In [12]:
vel_bc = velm - np.nanmin(velm)
acc_bc = np.nansum(vel_bc*ds*dx*1.e+3)
print acc_bc/1.e+6

101.091593136
